# 1.下载数据

In [1]:
import csv,xlrd,xlwt,datetime,time,re
import numpy as np

import pandas as pd


#一、导入指导价

#二、打开对方清单
def suanqingdan():
    suanqingdan=0
    global url
    jia_df=pd.read_excel('D:\20190409部分\20180929桌面备份\报买断业务\转贴现买断价格计算\jia.xls',encoding='GB18030')
    shijian0=time.strftime('%Y-%m-%d',time.localtime(time.time()))
    shijian=datetime.datetime.strptime(shijian0 , "%Y-%m-%d")#转为日期格式

#2.1输入交易日
    jin=input('请输入要交易日,输入格式为“2017-11-01”，不输入默认为今天:')
    if jin!='':
        shijian=datetime.datetime.strptime(jin , "%Y-%m-%d")
    jia_df['生效截至日'] = pd.to_datetime(jia_df['生效截至日'])#.astype('str')   #,format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式 注意要换成string格式
    jia_df=jia_df[((jia_df['生效截至日']>=shijian) | (jia_df['生效截至日']==''))&(jia_df['银行分类']!='BBB-及以下银行')]
    jia_df= jia_df.reset_index(drop=True)    #重新定义索引
    qixian1=jia_df.columns[2].split('至')
    qixian2=jia_df.columns[3].split('至')
    qixian3=jia_df.columns[4].split('至')
    qixian4=jia_df.columns[5].split('至')
    qixian5=jia_df.columns[6].split('至')

    qixian1[0] = datetime.datetime.strptime(qixian1[0] , "%Y-%m-%d")
    qixian1[1] = datetime.datetime.strptime(qixian1[1] , "%Y-%m-%d")
    qixian2[0] = datetime.datetime.strptime(qixian2[0] , "%Y-%m-%d")
    qixian2[1] = datetime.datetime.strptime(qixian2[1] , "%Y-%m-%d")
    qixian3[0] = datetime.datetime.strptime(qixian3[0] , "%Y-%m-%d")
    qixian3[1] = datetime.datetime.strptime(qixian3[1] , "%Y-%m-%d")
    qixian4[0] = datetime.datetime.strptime(qixian4[0] , "%Y-%m-%d")
    qixian4[1] = datetime.datetime.strptime(qixian4[1] , "%Y-%m-%d")
    qixian5[0] = datetime.datetime.strptime(qixian5[0] , "%Y-%m-%d")
    qixian5[1] = datetime.datetime.strptime(qixian5[1] , "%Y-%m-%d")


#2.2确认列名，到时候可以删除
    jixu=input('请价差列名是否有：出票日（日期格式）、承兑行、承兑行号、到期日（日期格式）、票面金额、票号，请回车确认:')
    if jixu!='':
        stop
    qing=pd.read_excel(url)

    print(qing)
#二、换列标题
    for i in range(0,len(qingcolumn)):
        for j in range(0,len(qingcolumn_df)):
            zhaop2= re.search(qingcolumn_df.loc[j,'原列名'] ,qingcolumn[i])
 
            if zhaop2:
                qing.rename(columns={qingcolumn[i]:qingcolumn_df.loc[j,'新列名']}, inplace=True)
                
                break

    if '贴现行' not in qing.columns.values.tolist():
        qing['贴现行']=''


#三、筛选列，计算清单
    qing=qing[['票号','票面金额','出票人','承兑行','贴现行','出票日','到期日']]
    qing= qing.reset_index(drop=True)    #重新定义索引
    for i in range(0,len(qing)):
        qing.loc[i,'承兑行号']=str(qing.loc[i,'票号'])[1:13]
    print(qing)
    qing['到期日2'] = pd.to_datetime(qing['到期日'])   #把str转化为时间格式，format='%Y%m%d'  吧yyyy-m-d 转化为yyyy-mm-dd的时间格式
    qing= qing.reset_index(drop=True)    #重新定义索引
    qing['承兑行号']=qing['承兑行号'].astype(str)
    qing['出票日']=qing['出票日'].astype(str)
    qing['到期日']=qing['到期日'].astype(str)

    for i in range (0,len(qing)):     
         if (qing.loc[i,'到期日2']>=qixian1[0]) and (qing.loc[i,'到期日2']<=qixian1[1]) :
           for j in range (0,len(jia_df)):
               if (jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']) and (float(jia_df.loc[j,jia_df.columns[2]])>=1) :
                   qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[2]]
                   qing.loc[i,'指导期限']=jia_df.columns[2]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
               
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')

                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian2[0]) and (qing.loc[i,'到期日2']<=qixian2[1]) :

           for j in range (0,len(jia_df)):

               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']    and (float(jia_df.loc[j,jia_df.columns[3]])>=1) :
                   
                   qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[3]]
                   qing.loc[i,'指导期限']=jia_df.columns[3]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]
                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian3[0]) and (qing.loc[i,'到期日2']<=qixian3[1])  :  #

           for j in range (0,len(jia_df)):
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行']  and (float(jia_df.loc[j,jia_df.columns[4]])>=1):
                   qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[4]]

                   qing.loc[i,'指导期限']=jia_df.columns[4]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1

         elif (qing.loc[i,'到期日2']>=qixian4[0]) and (qing.loc[i,'到期日2']<=qixian4[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[5]])>=1):
                   qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[5]]
                   qing.loc[i,'指导期限']=jia_df.columns[5]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1




         elif (qing.loc[i,'到期日2']>=qixian5[0]) and (qing.loc[i,'到期日2']<=qixian5[1])  :

           for j in range (0,len(jia_df)) :
               if jia_df.loc[j,'银行类型'] in qing.loc[i,'承兑行'] and (float(jia_df.loc[j,jia_df.columns[6]])>=1):
                   qing.loc[i,'邮储指导价']=jia_df.loc[j,jia_df.columns[6]]
                   qing.loc[i,'指导期限']=jia_df.columns[6]
                   qing.loc[i,'银行分类']=jia_df.loc[j,'银行分类']
                   qing.loc[i,'交易日']=str(shijian)
                   qing.loc[i,'交易日']=qing.loc[i,'交易日'].replace(' 00:00:00','')
                   qing.loc[i,'出票日']=qing.loc[i,'出票日'].replace(' 00:00:00','')
                   qing.loc[i,'到期日']=qing.loc[i,'到期日'].replace(' 00:00:00','')
                   qing.loc[i,'到期月']=qing.loc[i,'到期日'][:(len(qing.loc[i,'到期日'])-3)]

                   qing.loc[i,'承兑行号']=qing.loc[i,'承兑行号'].replace('.0','')
               
                   shijiancha=str(qing.loc[i,'到期日2']-shijian) #转化为str
                   shijiancha=int(shijiancha.replace(' days 00:00:00',''))  #转化为int
                   qing.loc[i,'期限']=shijiancha
                   qing.loc[i,'ji1']=shijiancha*qing.loc[i,'票面金额']
                   qing.loc[i,'ji2']=shijiancha*qing.loc[i,'票面金额']*qing.loc[i,'邮储指导价']
                   qing.loc[i,'可用额度']=jia_df.loc[j,'可用额度']
                   qing.loc[i,'生效截至日']=jia_df.loc[j,'生效截至日']
                   qing.loc[i,'授信有效']=jia_df.loc[j,'授信有效']
                   qing.loc[i,'我行评级']=jia_df.loc[j,'我行评级']
                   suanqingdan=suanqingdan+1
                   
    if suanqingdan<=0:
        print('整个清单都不能收！')
    else:
        qing['加权利率']=qing.loc[:,'ji2'].sum()/qing.loc[:,'ji1'].sum()
        del qing['到期日2']



        qing=qing[['票号','票面金额','出票人','承兑行','承兑行号','出票日','到期日','交易日','银行分类','邮储指导价','指导期限','加权利率','贴现行','可用额度','我行评级','生效截至日','授信有效','期限','到期月']]
#三、将算好的清单写入excel
        qing2=qing.replace(np.nan,99)
        print(qing2)
        qing2=qing2[qing2['邮储指导价'] != 99]  #排除NAN
        chengdui=qing2['票面金额'].groupby(qing2['银行分类']).sum()/10000
        print('  ')

        zongjin=qing2['票面金额'].sum()/10000
        print('总金额：%s'%zongjin)
        print('  清单分类如下：')

        print(chengdui)
        print('  ')
        yuefen=qing2['票面金额'].groupby(qing2['到期月']).sum()/10000
        print(yuefen)
        print('  ')
        qixian=qing2['期限'].mean()
        print(qixian)
        print('  ')


    
    
        url2=shijian0+duifang+'算加权.xls'
        writer = pd.ExcelWriter(url2)
        qing.to_excel(writer,'Sheet1')
        writer.save()





#四、制作导
        qing2['导汇票']=1
        qing2['导出票人账号']=''
        qing2['导收款人名称']=''
        qing2['导1']=''
        qing2['导2']=''
        qing2['导3']=''
        qing2['同城']=1
        qing2=qing2[['导汇票','票号','票面金额','出票日','到期日','出票人','导出票人账号','承兑行号','承兑行','导收款人名称','导1','导2','导3','承兑行号','同城','交易日','期限']]



        url3=shijian0+duifang+'导入.xls'
        writer = pd.ExcelWriter(url3)
        qing2.to_excel(writer,'Sheet1')
        writer.save()

for i in range(0,10):
    duifang=input('请输入文件名：')
 #   duifang=''+duifang
    url=('D:\20190409部分\20180929桌面备份\报买断业务\临时清单\\%s'%duifang)
    if duifang=='':
        continue
    else:
        suanqingdan()
    


请输入文件名：2019-04-252019-04-254.25出票 - 副本.xlsx算加权.xls导入.xls


FileNotFoundError: [Errno 2] No such file or directory: 'D:\x8190409部分\x8180929桌面备份\\报买断业务\\转贴现买断价格计算\\jia.xls'